# 텍스트 기반 추천

## 필요 라이브러리 

In [1]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

 __ai허브데이터 xlsx라 csv로 변환과정__ 
!pip install xlrd

!pip install openpyxl

xlsx = pd.read_excel('C:/Users/namva/자연어/dataset/감성대화/원천데이터//sentimenttalk.xlsx')

xlsx.to_csv('C:/Users/namva/자연어/dataset/감성대화/원천데이터//sentimenttalk.xlsx')

## 데이터 불러오기

In [4]:
data=pd.read_csv('C:/Users/namva/자연어/dataset/감성대화/원천데이터//sentimenttalk.xlsx')
df=data[ ['감정_대분류','감정_소분류','사람문장1','기계문장1','사람문장2','기계문장2','사람문장3','기계문장3']]

# 감정과 사람의 문장만 있게 변경
person = df[['감정_대분류','감정_소분류','사람문장1','사람문장2','사람문장3']]

## tf-idf 유사도 구하기

In [5]:
tfidf_vec = TfidfVectorizer(min_df=1) 
docs = person['사람문장1'].to_numpy()

In [6]:
cosine_similarity

array(['오늘 반장 선거에서 내가 반장이 됐어! 친구들이 날 믿어줘서 너무 고맙다.',
       '오늘 시험에 친구가 알려준 문제가 나왔어. 결과가 좋아서 기쁘다.',
       '오늘 대학 합격 통지를 받았어. 엄마가 너무 기뻐하는데 나는 눈물이 났어.', ...,
       '친구가 돈이 행복의 전부가 아니라고 하더니 노후 준비를 전혀 안 한 거야. 어휴 정말 걱정스러워.',
       '아들이 갑자기 찾아왔는데 손자가 희귀병이래. 너무 두려워.',
       '친구들에게는 자식이 선물해 줬다고 했지만 사실은 다 내가 구입한 거야.'], dtype=object)

In [8]:
tfidf_vec.fit(docs)

TfidfVectorizer()

In [9]:
tfidf_vec.idf_

array([7.11939791, 9.51729319, 8.82414601, ..., 9.51729319, 6.59952245,
       9.51729319])

In [10]:
tfidf_vec = TfidfVectorizer(min_df=1) 
docs = person['사람문장1'].to_numpy() # series -> numpy, array
tfidf_mat = tfidf_vec.fit_transform(docs) # tfidf matrix
docs_similarties = (tfidf_mat * tfidf_mat.T) # tfidf 유사도 식
# print(docs_similarties.toarray())

In [11]:

docs_similarties

<10000x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 10102571 stored elements in Compressed Sparse Row format>

## 코사인 유사도 및 데이터 변환

In [12]:
cosine_mat= cosine_similarity(tfidf_mat, tfidf_mat) # tf-idf 유사도 -> cosine 유사도
np.round(cosine_mat, 4) 

# index기반 구하기
idx2title={}
for i,c in enumerate(person['사람문장1']): idx2title[i] = c
# title기반 구하기
title2idx={}
for i,c in idx2title.items(): title2idx[c]= i

## 실행과정(입력 -> 유사도 -> 상위 10개의 유사도 출력)

In [13]:
# 입력
idx= title2idx['친구가 돈이 행복의 전부가 아니라고 하더니 노후 준비를 전혀 안 한 거야. 어휴 정말 걱정스러워.']
# 동일한 텍스트 제외 유사도 구하기, 정렬
sim_score = [(i,c) for i,c in enumerate(cosine_mat[idx]) if i != idx]
sim_score = sorted(sim_score, key=lambda x:x[1],reverse=True)
# 상위 10개 추출
sim_score = [(idx2title[i],c) for i,c in sim_score[:10]]
sim_score

[('노후 준비를 잘해놓은 데다가 몸도 건강해져서 노후가 여유로워.', 0.17653082836237577),
 ('노후 준비를 잘해놓은 데다가 몸도 건강해져서 노후가 여유로워.', 0.17653082836237577),
 ('나 건강때문에 걱정스러워.', 0.16774696363010796),
 ('친구가 고혈압으로 쓰러져서 입원했는데 너무 걱정스러워.', 0.15664592048689757),
 ('언니가 취업 준비 힘들게 하더니 드디어 공채 시험에 합격했어! 정말 기뻐.', 0.13391718525025956),
 ('늙으면 죽어야 해. 다 늙어서 저게 무슨 추태야 어휴 역겨워.', 0.13246316165442582),
 ('돈이 좀 넉넉하면 좋겠는데 이럴 땐 돈이 없어서 속상하다.', 0.13239035367189944),
 ('세상에 공부가 전부가 아닌데 선생님께서 공부 문제로 심하게 말씀하셔서 마음이 아파.', 0.13192627736403864),
 ('친구가 나한테는 시간 없다고 하더니 다른 친구들하고 놀러 갔더라. 너무 우울해.', 0.13167440191015647),
 ('오해를 해서 화가 많이 난 친구가 내 말을 전혀 들으려고 하지 않아.', 0.13135974450816382)]

In [107]:
tfidf_vec.transform(["안녕하세요."])

<1x20108 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Row format>

In [108]:
voc = tfidf_vec.vocabulary_
vo

{'오늘': 12857,
 '반장': 7196,
 '선거에서': 9625,
 '내가': 3101,
 '반장이': 7202,
 '됐어': 4760,
 '친구들이': 17732,
 '믿어줘서': 6978,
 '너무': 3238,
 '고맙다': 1289,
 '시험에': 10596,
 '친구가': 17703,
 '알려준': 11759,
 '문제가': 6757,
 '나왔어': 2825,
 '결과가': 1059,
 '좋아서': 15801,
 '기쁘다': 2237,
 '대학': 4278,
 '합격': 19162,
 '통지를': 18166,
 '받았어': 7268,
 '엄마가': 12325,
 '기뻐하는데': 2224,
 '나는': 2676,
 '눈물이': 3604,
 '났어': 3060,
 '졸업식인데': 15740,
 '선생님께서': 9668,
 '아프셔서': 11441,
 '오지': 12961,
 '못하셨어': 6588,
 '그동안': 1960,
 '감사하다는': 411,
 '말도': 5892,
 '못했는데': 6607,
 '올림피아드에서': 13033,
 '금상': 2091,
 '고생이': 1342,
 '헛되지': 19459,
 '않아서': 11680,
 '기뻐': 2221,
 '요즘': 13264,
 '부모님': 8114,
 '부부싸움': 8169,
 '때문에': 5354,
 '많이': 5862,
 '힘들었는데': 20092,
 '친구': 17702,
 '미영이': 6912,
 '덕분에': 4365,
 '기분이': 2216,
 '한결': 19038,
 '나아졌어': 2765,
 '학교에서': 18959,
 '친구랑': 17749,
 '다퉜는데': 3901,
 '선생님은': 9675,
 '나만': 2704,
 '뭐라고': 6852,
 '하셔': 18901,
 '준비물을': 16109,
 '깜빡': 2393,
 '했는데': 19349,
 '민우가': 6950,
 '쉬는': 10295,
 '시간에': 10429,
 '뛰어가서': 5514,
 '사다': 8793,
 '있지